In [6]:
from selenium import webdriver
import numpy as np
import glob
import os
import datetime
import pandas as pd
import re
from bs4 import BeautifulSoup

### Scrapping website

In [2]:
d = webdriver.PhantomJS()
d.get("https://bitinfocharts.com/")

In [4]:
webdriver.PhantomJS()

<selenium.webdriver.phantomjs.webdriver.WebDriver (session="5ce6a3a0-cd7b-11e7-8fd7-5fe91f0fc5b2")>

In [3]:
table = d.find_element_by_id('main_body').find_element_by_tag_name('tbody').get_attribute('innerHTML')
bs = BeautifulSoup(table, "lxml")
d.quit()

In [33]:
df = pd.DataFrame()
df['Columns'] = [k.find('td').getText() for k in bs.find_all('tr')]
for crypto in ['btc', 'eth', 'ltc']:
    df[crypto.upper()] = [p.getText() for p in bs.find_all(class_='c_%s' % crypto)]

### Cleaning dataframe

In [36]:
df2 = df.copy()
stopwords = ['BTC', 'ETH', 'LTC']
def remove_chars(x):
    num = re.sub('[^0-9]','', x)
    if len(str(num)) < 3:
        return x
    else:
        return num
    
def convert_number(x):
    return float(x.replace(',',''))

def remove_crypt(x):
    for k in stopwords:
        if k in x:
            return convert_number(x[:x.index(k)])
        
def clean_reward(x):
    for k in stopwords:
        if k in x:
            x = x[:x.index(k)]
            return tuple(map(float, x.replace(',','').split('+')))

df2.loc[0] = df2.loc[0].apply(lambda x:x[:x.index('(')] if '(' in x else x)
df2.loc[1] = df2.loc[1].apply(remove_chars)
df2.loc[2] = df2.loc[2].apply(lambda x:convert_number(x[x.find('$')+1:x.find('USD')]) if 'USD' in x else x)
df2.loc[3] = df2.loc[3].apply(lambda x:convert_number(x[x.find('$')+1:x.find('USD')]) if 'USD' in x else x)
df2.iloc[4, 1:] = df2.iloc[4, 1:].apply(convert_number)
for p in range(6,10):
    df2.iloc[p, 1:] = df2.iloc[p, 1:].apply(remove_crypt)
df2.iloc[14, 1:] = df2.iloc[14, 1:].apply(clean_reward)
df2.iloc[15, 1:] = df2.iloc[15, 1:].apply(clean_reward)

In [37]:
df2.head(20)

,Columns,BTC,ETH,LTC
0,,Bitcoin,Ethereum,Litecoin
1,Total,16676336,95830620,53902108
2,Price,7970.95,357.54,71.34
3,Market Capitalization,1.32926e+11,3.42632e+10,3.8453e+09
4,Transactions last 24h,262628,466537,23654
5,Transactions avg. per hour,"10,943","19,439",986
6,Sent last 24h,5.31232e+06,1.26271e+07,1.71879e+07
7,Sent avg. per hour,221347,526129,716164
8,Avg. Transaction Value,20.23,27.07,726.64
9,Median Transaction Value,0.058,0.0093,5.1
